# Signal Research Template

Standard workflow: load data → compute signals → single-asset backtest → multi-asset portfolio backtest → evaluate → save to registry if promising.

## 1. Setup

In [ ]:
import sys
import pandas as pd
from pathlib import Path

project_root = Path().resolve()
if project_root.name == 'notebooks':
    project_root = project_root.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from research.helpers import load_prices, load_fred, evaluate_signal, plot_backtest
from research.signals import MomentumSignal, CarrySignal, MeanReversionSignal, get_signal, list_signals, register

## 2. Data Loading

In [ ]:
# Load prices from Parquet data lake (run data update first if empty)
asset_class = "equities"
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META"]
start, end = "2020-01-01", "2023-12-31"

prices = load_prices(asset_class, tickers, start, end)
print(f"Loaded {prices.shape[0]} rows, {prices.shape[1]} tickers")
prices.head()

In [ ]:
# Optional: load FRED macro data for regime/context
# fred_df = load_fred("macro_indicators", ["UNRATE", "CPIAUCSL"], start, end)
# fred_df.head()

## 3. Signal Computation

In [ ]:
# Compute signals from registry or create custom
momentum = MomentumSignal(lookback=252, skip=21)
signal_scores = momentum.compute(prices)
positions = momentum.to_positions(signal_scores)

# For single-asset backtest, use one column
single_ticker = tickers[0]
pos_series = positions[single_ticker] if isinstance(positions, pd.DataFrame) else positions
price_series = prices[single_ticker]
print(f"Signal for {single_ticker}: {pos_series.dropna().tail(3).tolist()}")

## 4. Single-Asset Backtest (Vectorized)

In [ ]:
result = evaluate_signal(pos_series, price_series, cost_bps=10)
print("Stats:", result.stats)
plot_backtest(result).show()

## 5. Multi-Asset Portfolio Backtest

In [ ]:
from backtests.portfolio_backtest import run_portfolio_backtest

# Build blended alpha from multiple signals
momentum_sig = MomentumSignal(lookback=252, skip=21)
mr_sig = MeanReversionSignal(lookback=63)

port_result = run_portfolio_backtest(
    prices=prices,
    signals=[momentum_sig, mr_sig],
    signal_weights=[1.0, 0.5],
    cost_bps=10,
)
print("Portfolio stats:", port_result.stats)
plot_backtest(port_result).show()

## 6. Statistical Evaluation

In [ ]:
stats = result.stats
print(f"Sharpe: {stats['sharpe']:.3f}")
print(f"Max Drawdown: {stats['max_drawdown']:.2%}")
print(f"Total Return: {stats['total_return']:.2%}")
print(f"Vol (daily): {stats['vol_daily']:.4f}")

## 7. Save to Signal Registry (if promising)

In [ ]:
# If your custom signal performs well, add it to the registry:
# from research.signals import BaseSignal, register
#
# class MyCustomSignal(BaseSignal):
#     name = "my_custom"
#     def compute(self, prices):
#         return ...  # your logic
#
# register(MyCustomSignal())
# print("Registered:", list_signals())